In [6]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"

In [7]:
number_of_shots = 7

In [8]:
nbit = 4
llamasize = '7B'
# llamasize = '13B'
temp = 0.1

In [9]:
experiment_name = "2023.12.06-03_LLama2_7B-chat-hf_4-bit_float-16_7-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [10]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"

top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

output_data_path = base_path + "/../Llama-2/data/output/" + experiment_name
log_filename = base_path + "/../Llama-2/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


# LOGIN DA HUGGING FACE

In [11]:
# import os
# from IPython.display import clear_output
# os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
# clear_output()

from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [12]:
# @title
!huggingface-cli login --token $huggingface_cli_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
# @title
# from pathlib import Path

In [14]:
# @title
# # import logging

# # logger = logging.getLogger('my_logger')
# # # , level=logging.INFO,
# # #logging.basicConfig(filename=base_path + f"/../Llama-2/logs/{experiment_name}.log", filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')
# # logging.basicConfig(filename="test.log", filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')

# %%python

# import logging
# logger = logging.getLogger('my_logger')

# logging.basicConfig(
#     #filename=experiment_name+'.log', # write to this file
#     filename = 'app.log',
#     filemode='a', # open in append mode
#     level=logging.INFO,
#     format='%(name)s - %(levelname)s - %(message)s'
#     )

# # logging.warning('This will get logged to a file')
# logging.info('test')

In [15]:
# @title
# print(Path('app.log').read_text())

In [16]:
# @title
# from google.colab import files

# !mv app.log output_data_path

In [17]:
# @title
# import locale
# print(locale.getpreferredencoding())

In [18]:
# @title
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [19]:
# @title
# !touch /content/gdrive/MyDrive/Colab/SDG/data/../Llama-2/logs/2023.12.06-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES.log

In [20]:
# @title
# base_path + f"/../Llama-2/logs/{experiment_name}.log"

In [21]:
# @title
# output_data_path

In [22]:
# @title
# logging.info("test")
# logging.warning('This will get logged to a file')

In [23]:
# @title
# print(Path(base_path + f"/../Llama-2/logs/{experiment_name}.log").read_text())
# print(Path("test.log").read_text())

# INSTALLAZIONE E IMPORT LIBRERIE

In [24]:
# @title
import pickle

In [25]:
# @title
import warnings
warnings.filterwarnings("ignore")

In [26]:
# @title
with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [27]:
# @title
#!pip install --upgrade pip
!pip install -q transformers --progress-bar off # Download e caricamento modelli
!pip install -q bitsandbytes --progress-bar off # Quantizzazione
!pip install -q accelerate --progress-bar off

In [28]:
# @title
# Installazione "permanente"
!pip install -q sentencepiece --progress-bar off && touch ~/.ipython/profile_default/startup/00-first.py && echo "import <sentencepiece>" >> ~/.ipython/profile_default/startup/00-first.py

In [29]:
# @title
!pip install -q pillow==10.0.1 --progress-bar off
!pip install -q langchain[llms] --progress-bar off

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [30]:
# @title
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [31]:
# @title
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [32]:
# @title
torch.cuda.empty_cache()

# DOWNLOAD E CARICAMENTO DEL MODELLO

In [33]:
# @title
# pip install --upgrade bitsandbytes

In [34]:
# @title
model = LlamaForCausalLM.from_pretrained(  # Caricamento del modello usando la libreria dedicata. In questo caso, "ForCausal" si riferisce al modello per la generazione di testo
    model_id, # Nome del modello precedentemente impostato
    load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
    torch_dtype = torch.float16 # Impostare questa codifica aiuta il modello in fase di generazione, in quanto il prompt è tipicamente float16
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [35]:
# @title
torch.cuda.empty_cache()

In [36]:
# @title
# !nvidia-smi

In [37]:
# @title
tokenizer = LlamaTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [38]:
# @title
torch.cuda.empty_cache()

In [39]:
# @title
# model.eval()

In [40]:
# @title
torch.cuda.empty_cache()

## TEST

In [41]:
# @title
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [42]:
# @title
torch.cuda.empty_cache()

In [43]:
# @title
llm = HuggingFacePipeline(pipeline=generate_text)

In [44]:
# @title
torch.cuda.empty_cache()

In [45]:
# @title
data = pd.read_excel(input_data_path)

In [46]:
# @title
# data.shape

In [47]:
# @title
# data.groupby('sdg').count()['text']

In [48]:
# @title
data_sel = data

In [49]:
# @title
# import pandas as pd
# data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(2)
# data_sel_4_few_shot_examples

In [50]:
# @title
data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])

In [51]:
# @title
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(7)

# data_sel_4_few_shot_examples

In [52]:
# @title
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'

#     few_shots += input_text
#     few_shots += output_text

# print(few_shots)

In [53]:
# @title
# template = """[INST]
# <<SYS>>
# Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""


# The text to be processed is:
# <</SYS>>

# '''
# {text}
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [54]:
# @title
# prompt = PromptTemplate(
#     input_variables=["text"],
#     template=template,
# )

In [55]:
# @title
torch.cuda.empty_cache()

In [56]:
# @title
# top7_train_data_sim_to_each_test_data

In [57]:
# file = open(log_filename, 'a+')
# file.write("ecchecaz")
# file.close()

# log_filename

In [77]:
data_sel = data.iloc[147:]

In [78]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 147
results = []

for index, row in data_sel.iterrows():

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')
  file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  print(few_shots)
  file.write(few_shots)
  file.write('\n')


  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
{text}
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""
  try:

    prompt = PromptTemplate(
      input_variables=["text"],
      template=template,
    )

    # print(template)

    result = llm(prompt.format(text=text))
  except Exception as e:
    print(cnt, text_id, sdg_true, e)
    result = str(e)
    output_text = "Si è verificato un errore: \n("+str(cnt)+", "+str(text_id)+", "+str(sdg_true)+", "+str(e)+")\n"
    file.write(output_text)

  result_extended = (cnt, text_id, sdg_true, result)

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_Llama2_7B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1
  file.close()
  # if cnt==5:
  #   break



Output streaming troncato alle ultime 5000 righe.
Output: "---SDG 6---"

Input text: "Time-use data should be taken into account to incorporate the gender perspective when designing poverty-eradication programmes, or when evaluating the benefits of such conditions, and to promote shared responsibility between men and women and between the State, the market and families (ECLAC, 2016a), and when coordinating these programmes with others and strengthening health and education policies so that they reach the poorest sectors and are not supported by conditional transfer programmes and the unpaid work of women. The reasons vary, and include the cost and organization of health services, which complicate access to medical care, especially for people in the lowest-income households (Ferran, 2008). In order to produce estimates of the activities that directly affect the health of household members, time-use surveys must distinguish between care activities relating specifically to health care and

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Output streaming troncato alle ultime 5000 righe.

Input text: "In addition, it is regulated that the umbrella organisations provide support to schools via their PBDs. Furthermore, networks and umbrella organisations are key players in the VLOR. This position makes the umbrella organisations indispensable in the development and implementation of attainment targets."
Output: "---SDG 4---"


(847, 432, 14, 'Based on the given text, I would classify it under SDG 14 - Life Below Water. The text primarily deals with the sustainable management of coastal fisheries and the protection of the marine environment, which aligns with the goals of SDG 14.')


##################################################
EXPECTED_SDG: 14 TEXT: The drawbacks of this single-species management approach have led to calls for an ecosystem based approach to fishery management, but, for most fisheries, this has yet to be operationalized in terms of setting the fishery reference points that dictate rebuilding requireme

In [79]:
# @title
# data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==666]

In [80]:
# @title
# data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==2729]

In [81]:
# @title
# data_sel_4_few_shot_examples

In [92]:
data_sel = data

In [93]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

1020

In [94]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

,cnt,text_id,sdg_true,result
0,0,6668,0,---SDG 0---
1,1,6099,0,---SDG 1---
2,2,6471,0,---SDG 16---


In [95]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed.head(3)

,cnt,text_id,sdg_true,result,label
0,0,6668,0,---SDG 0---,0
1,1,6099,0,---SDG 1---,1
2,2,6471,0,---SDG 16---,16


In [96]:
final = data_sel.merge(results_processed, left_index=True, right_index=True)
final.head(3)

,Unnamed: 0,text_id_x,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,text_id_y,sdg_true,result,label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,---SDG 0---,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 1---,1
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---,16


In [97]:
final.to_excel(output_data_path+f"/{experiment_name}.xlsx")

In [98]:
final  = final[~final.label.isna()]

In [99]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1012
1012
              precision    recall  f1-score   support

           0       0.56      0.15      0.24        59
           1       0.51      0.65      0.57        60
           2       0.86      0.32      0.47        59
           3       0.71      0.93      0.81        60
           4       0.81      0.80      0.81        60
           5       0.93      0.63      0.75        60
           6       0.78      0.86      0.82        58
           7       0.73      0.75      0.74        60
           8       0.35      0.59      0.44        59
           9       0.57      0.67      0.62        60
          10       0.59      0.37      0.46        59
          11       0.67      0.50      0.57        60
          12       0.60      0.82      0.70        60
          13       0.75      0.60      0.67        60
          14       0.89      0.65      0.75        60
          15       0.62      0.83      0.71        59
          16       0.61      0.78      0.69        59
          17     

In [100]:
classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

'/content/gdrive/MyDrive/Colab/SDG/data/../Llama-2/data/output/2023.12.06-03_LLama2_7B-chat-hf_4-bit_float-16_7-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES/2023.12.06-03_LLama2_7B-chat-hf_4-bit_float-16_7-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES_classification_report.xlsx'

In [101]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

In [102]:
test = pd.read_excel(classification_report_filename)
test

,Unnamed: 0,precision,recall,f1-score,support
0,0,0.562500,0.152542,0.240000,59.000000
1,1,0.513158,0.650000,0.573529,60.000000
2,2,0.863636,0.322034,0.469136,59.000000
3,3,0.708861,0.933333,0.805755,60.000000
4,4,0.813559,0.800000,0.806723,60.000000
5,5,0.926829,0.633333,0.752475,60.000000
6,6,0.781250,0.862069,0.819672,58.000000
7,7,0.725806,0.750000,0.737705,60.000000
8,8,0.346535,0.593220,0.437500,59.000000
9,9,0.571429,0.666667,0.615385,60.000000
